In [ ]:
import pandas as pd

# Load the CSV with error handling for quotes
input_file = "/content/resource-neo4j_import-v2.csv"  # Replace with your actual file name
output_file = "/content/resource-neo4j_import-v2-cleaned.csv"




# Read CSV safely by enforcing proper delimiter handling
df = pd.read_csv(input_file, dtype=str, engine="python")

# Remove problematic quotes and extra spaces
df = df.applymap(lambda x: x.strip().replace('"', '').replace(',', '') if isinstance(x, str) else x)

# Ensure column count is correct
expected_columns = ["npi", "first_name", "last_name", "gender", "credential",
                    "specialty", "facility", "city", "state", "zip_code", "year","med_school"]

# Keep only expected columns
df = df[expected_columns]

# Save the cleaned CSV
df.to_csv(output_file, index=False)

print(f" Cleaned CSV saved as: {output_file}")



In [ ]:
df["year"] = pd.to_numeric(df["year"], errors="coerce")

# Count total number of records
total_records = len(df)

# Group by `grad_year` and count occurrences
grad_year_counts = df.groupby("year").size().reset_index(name="count")

# Display results
print(f"Total Records: {total_records}")
print("Records Grouped by Graduation Year:")
print(grad_year_counts)


In [ ]:
df_filtered = df[(df["year"] >= 2015) & (df["year"] <= 2017)]

In [ ]:
# Replace NaN values with "UNKNOWN" for Facility name
df_filtered["facility"].fillna("UNKNOWN", inplace=True)
df_filtered["city"].fillna("UNKNOWN", inplace=True)
df_filtered["state"].fillna("UNKNOWN", inplace=True)
df_filtered["zip_code"].fillna("UNKNOWN", inplace=True)

# Drop rows where facility is still missing
df_filtered = df_filtered[df_filtered["facility"] != "UNKNOWN"]

# Count job movements per doctor (based on unique facility entries)
job_mobility_counts = df_filtered.groupby("npi")["facility"].nunique() - 1

# Ensure no negative values (new doctors may have 0 moves)
job_mobility_counts = job_mobility_counts.clip(lower=0)

# Merge back to df_filtered
df_filtered["job_moves"] = df_filtered["npi"].map(job_mobility_counts)



In [ ]:
!pip install neo4j

In [ ]:
from neo4j import GraphDatabase

URI = ""
AUTH = ("neo4j", "")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [ ]:
CSV_PATH = "/content/resource-neo4j_import-v2-cleaned.csv"  # Update file path
df = pd.read_csv(CSV_PATH, dtype=str)


df["year"] = pd.to_numeric(df["year"], errors="coerce")
df = df.dropna(subset=["npi", "facility", "year", "specialty"])


doctor_credentials = {"MD", "DO", "DPM", "DDS"}
df_filtered = df[df["credential"].isin(doctor_credentials)].drop_duplicates()

df_filtered["facility"] = df_filtered["facility"].str.strip().str.lower()
unique_facilities_df = df_filtered[["facility", "city", "state", "zip_code"]].drop_duplicates()


df_filtered["med_school"] = df_filtered["med_school"].str.strip().str.lower()




In [ ]:

doctor_credentials = {"MD", "DO", "DPM", "DDS"}

df_filtered = df[df["credential"].isin(doctor_credentials)]

print(f" Remaining Doctors in Data: {df_filtered.shape[0]}")
print(f" Unique Credentials After Filtering: {df_filtered['credential'].unique()}")


In [ ]:
from tqdm import tqdm


print(f" Unique Doctors: {df_filtered['npi'].nunique()}")
print(f" Unique Facilities: {unique_facilities_df.shape[0]}")

def create_doctor(tx, row):
    query = """
    MERGE (d:Doctor {npi: $npi})
    SET d.first_name = $first_name,
        d.last_name = $last_name,
        d.gender = $gender,
        d.credential = $credential,
        d.specialty = $specialty;
    """
    tx.run(query,
           npi=row["npi"], first_name=row["first_name"],
           last_name=row["last_name"], gender=row["gender"],
           credential=row["credential"], specialty=row["specialty"])

def create_facility(tx, row):
    query = """
    MERGE (f:Facility {name: $facility})
    SET f.city = COALESCE($city, "UNKNOWN"),
        f.state = COALESCE($state, "UNKNOWN"),
        f.zip_code = COALESCE($zip_code, "UNKNOWN");
    """
    tx.run(query,
           facility=row["facility"], city=row["city"],
           state=row["state"], zip_code=row["zip_code"])


with driver.session() as session:
    print(" Inserting Doctor nodes into Neo4j...")
    for index, row in tqdm(df_filtered.drop_duplicates(subset=["npi"]).iterrows(), total=df_filtered["npi"].nunique(), desc="Processing Doctors"):
        session.execute_write(create_doctor, row)

print(" Doctor nodes successfully added to Neo4j!")


with driver.session() as session:
    print(" Inserting Facility nodes into Neo4j...")
    for index, row in tqdm(unique_facilities_df.iterrows(), total=unique_facilities_df.shape[0], desc="Processing Facilities"):
        session.execute_write(create_facility, row)

print(" Facility nodes successfully added to Neo4j!")

In [ ]:
def update_doctor_grad_year(tx, row):
    query = """
    MATCH (d:Doctor {npi: $npi})
    SET d.grad_year = toInteger($grad_year)
    """
    tx.run(query, npi=row["npi"], grad_year=row["year"])


df_grad_year = df_filtered[["npi", "year"]].dropna().drop_duplicates()

with driver.session() as session:
    print(" Updating Doctor nodes with graduation year...")
    for _, row in tqdm(df_grad_year.iterrows(), total=df_grad_year.shape[0], desc="Adding grad_year"):
        session.execute_write(update_doctor_grad_year, row)

print(" Graduation years successfully updated!")


In [ ]:
from tqdm import tqdm

def update_doctor_grad_year(tx, row):
    query = """
    MATCH (d:Doctor {npi: $npi})
    SET d.grad_year = toInteger($grad_year)
    """
    tx.run(query, npi=row["npi"], grad_year=row["year"])

df_grad_year = df_filtered[["npi", "year"]].dropna().drop_duplicates().reset_index(drop=True)

start_index = 58112
df_resume = df_grad_year.iloc[start_index:]

with driver.session() as session:
    print(f" Resuming Doctor grad_year updates from index {start_index}...")
    for idx, row in tqdm(df_resume.iterrows(), total=len(df_resume), desc="Resuming grad_year"):
        session.execute_write(update_doctor_grad_year, row)

print(" Remaining grad_year updates completed!")


In [ ]:
# Extract unique medical schools
unique_med_schools_df = df_filtered[["med_school"]].drop_duplicates()

# Print summary
print(f" Unique Medical Schools: {unique_med_schools_df.shape[0]}")

def create_medical_school(tx, row):
    query = """
    MERGE (m:MedicalSchool {name: $med_school});
    """
    tx.run(query, med_school=row["med_school"])





In [ ]:
# Ensure year is numeric and drop NaN values
df_filtered["year"] = pd.to_numeric(df_filtered["year"], errors="coerce")
df_filtered = df_filtered.dropna(subset=["year", "specialty"])

# Extract unique years
unique_years_df = df_filtered[["year"]].drop_duplicates()

# Extract unique job placements (year + specialty)
unique_job_placements_df = df_filtered[["year", "specialty"]].drop_duplicates()

# Print summary
print(f" Unique Years: {unique_years_df.shape[0]}")
print(f" Unique Job Placements: {unique_job_placements_df.shape[0]}")


In [ ]:
def create_year(tx, row):
    query = """
    MERGE (y:Year {value: $year});
    """
    tx.run(query, year=row["year"])

def create_job_placement(tx, row):
    query = """
    MERGE (j:JobPlacement {year: $year, specialty: $specialty});
    """
    tx.run(query, year=row["year"], specialty=row["specialty"])


with driver.session() as session:
    print(" Inserting Year nodes into Neo4j...")
    for index, row in tqdm(unique_years_df.iterrows(), total=unique_years_df.shape[0], desc="Processing Years"):
        session.execute_write(create_year, row)

print(" Year nodes successfully added to Neo4j!")

with driver.session() as session:
    print(" Inserting JobPlacement nodes into Neo4j...")
    for index, row in tqdm(unique_job_placements_df.iterrows(), total=unique_job_placements_df.shape[0], desc="Processing JobPlacements"):
        session.execute_write(create_job_placement, row)

print(" JobPlacement nodes successfully added to Neo4j!")



Relationships


In [ ]:
unique_works_at_df = df_filtered[["npi", "facility"]].drop_duplicates()
unique_has_participant_df = df_filtered[["npi", "year", "specialty"]].drop_duplicates()
unique_happens_at_df = df_filtered[["year", "specialty", "facility"]].drop_duplicates()
unique_has_time_reference_df = df_filtered[["year", "specialty"]].drop_duplicates()

job_mobility_counts = df_filtered.groupby("npi")["facility"].nunique() - 1
job_mobility_counts = job_mobility_counts.clip(lower=0)  # Ensure no negatives
df_filtered["job_moves"] = df_filtered["npi"].map(job_mobility_counts)
unique_has_mobility_df = df_filtered[["npi", "job_moves"]].drop_duplicates()

print(f" WORKS_AT Relationships: {unique_works_at_df.shape[0]}")
print(f" HAS_PARTICIPANT Relationships: {unique_has_participant_df.shape[0]}")
print(f" HAPPENS_AT Relationships: {unique_happens_at_df.shape[0]}")
print(f" HAS_TIME_REFERENCE Relationships: {unique_has_time_reference_df.shape[0]}")
print(f" HAS_MOBILITY Relationships: {unique_has_mobility_df.shape[0]}")


In [ ]:
def link_doctor_to_facility(tx, row):
    query = """
    MATCH (d:Doctor {npi: $npi})
    MATCH (f:Facility)
    WHERE toLower(f.name) = toLower($facility)
    MERGE (d)-[:WORKS_AT]->(f);
    """
    tx.run(query, npi=row["npi"], facility=row["facility"])




def link_doctor_to_jobplacement(tx, row):
    query = """
    MATCH (d:Doctor {npi: $npi})
    MATCH (j:JobPlacement)
    WHERE j.year = $year AND toUpper(j.specialty) = toUpper($specialty)
    MERGE (d)-[:HAS_PARTICIPANT]->(j);
    """
    tx.run(query, npi=row["npi"], year=row["year"], specialty=row["specialty"])


def link_jobplacement_to_facility(tx, row):
    query = """
    MATCH (j:JobPlacement)
    WHERE j.year = $year AND toUpper(j.specialty) = toUpper($specialty)
    MATCH (f:Facility)
    WHERE toLower(f.name) = toLower($facility)
    MERGE (j)-[:HAPPENS_AT]->(f);
    """
    tx.run(query, year=row["year"], specialty=row["specialty"], facility=row["facility"])


def link_jobplacement_to_year(tx, row):
    query = """
    MATCH (j:JobPlacement)
    WHERE j.year = $year AND toUpper(j.specialty) = toUpper($specialty)
    MATCH (y:Year {value: $year})
    MERGE (j)-[:HAS_TIME_REFERENCE]->(y);
    """
    tx.run(query, year=row["year"], specialty=row["specialty"])


def link_doctor_to_medical_school(tx, row):
    query = """
    MATCH (d:Doctor {npi: $npi})
    MATCH (m:MedicalSchool)
    WHERE toLower(m.name) = toLower($med_school)
    MERGE (d)-[:GRADUATED_FROM]->(m);
    """
    tx.run(query, npi=row["npi"], med_school=row["med_school"])





In [ ]:
from tqdm import tqdm

with driver.session() as session:
    print(" Creating WORKS_AT relationships...")
    for index, row in tqdm(unique_works_at_df.iterrows(), total=unique_works_at_df.shape[0], desc="Processing WORKS_AT"):
        session.execute_write(link_doctor_to_facility, row)

print(" WORKS_AT relationships successfully added to Neo4j!")





In [ ]:

with driver.session() as session:
    print(" Creating HAS_PARTICIPANT relationships...")
    for index, row in tqdm(unique_has_participant_df.iterrows(), total=unique_has_participant_df.shape[0], desc="Processing HAS_PARTICIPANT"):
        session.execute_write(link_doctor_to_jobplacement, row)

print(" HAS_PARTICIPANT relationships successfully added to Neo4j!")

In [ ]:
with driver.session() as session:
    print(" Creating MedicalSchool nodes...")
    for index, row in tqdm(unique_med_schools_df.iterrows(), total=unique_med_schools_df.shape[0], desc="Processing Medical Schools"):
        session.execute_write(create_medical_school, row)

print(" MedicalSchool nodes successfully added to Neo4j!")

In [ ]:
# 🚀 Insert HAPPENS_AT Relationships
with driver.session() as session:
    print(" Creating HAPPENS_AT relationships...")
    for index, row in tqdm(unique_happens_at_df.iterrows(), total=unique_happens_at_df.shape[0], desc="Processing HAPPENS_AT"):
        session.execute_write(link_jobplacement_to_facility, row)

print(" HAPPENS_AT relationships successfully added to Neo4j!")




In [ ]:
with driver.session() as session:
    print(" Creating HAS_TIME_REFERENCE relationships...")
    for index, row in tqdm(unique_has_time_reference_df.iterrows(), total=unique_has_time_reference_df.shape[0], desc="Processing HAS_TIME_REFERENCE"):
        session.execute_write(link_jobplacement_to_year, row)

print(" HAS_TIME_REFERENCE relationships successfully added to Neo4j!")



In [ ]:
from tqdm import tqdm  # Import tqdm for tracking

with driver.session() as session:
    print(" Creating GRADUATED_FROM relationships...")
    for index, row in tqdm(df_filtered[["npi", "med_school"]].drop_duplicates().iterrows(), total=df_filtered[["npi", "med_school"]].drop_duplicates().shape[0], desc="Processing GRADUATED_FROM"):
        session.execute_write(link_doctor_to_medical_school, row)

print(" GRADUATED_FROM relationships successfully added to Neo4j!")


In [ ]:
import pandas as pd

# Load ADI data
adi_df = pd.read_csv("/content/medical_doctors_with_ADI.csv", dtype=str)

# Standardize ZIP codes to 5-digit format
adi_df['5_digit_zip_code'] = adi_df['ZIP_Code_5_Digit_Standardized'].astype(str).str.zfill(5)

# Convert numeric fields to float
adi_df['adi_nat_2015'] = adi_df['Avg_ADI_NATRANK_2015'].astype(float)
adi_df['adi_state_2015'] = adi_df['Avg_ADI_STATERNK_2015'].astype(float)
adi_df['adi_nat_2020'] = adi_df['Avg_ADI_NATRANK_2020'].astype(float)
adi_df['adi_state_2020'] = adi_df['Avg_ADI_STATERNK_2020'].astype(float)
adi_df['gisjoin_2015'] = adi_df['GISJOIN_2015']
adi_df['gisjoin_2020'] = adi_df['GISJOIN_2020']

# Extract unique ADI scores by ZIP
adi_by_zip = adi_df.groupby("5_digit_zip_code").agg({
    'adi_nat_2015': 'mean',
    'adi_state_2015': 'mean',
    'adi_nat_2020': 'mean',
    'adi_state_2020': 'mean',
    'gisjoin_2015': 'first',
    'gisjoin_2020': 'first'
}).reset_index()

print(f" Extracted {len(adi_by_zip)} unique ZIP code ADI entries.")

# Standardize ZIP in df_filtered
df_filtered['5_digit_zip_code'] = df_filtered['zip_code'].astype(str).str[:5].str.zfill(5)

# Merge ADI data into your main dataframe
df_filtered = df_filtered.merge(adi_by_zip, on='5_digit_zip_code', how='left')

print(" ADI scores merged into df_filtered!")


In [ ]:
from tqdm import tqdm

# Prepare a unique list of facilities with ADI by 5-digit ZIP code
facility_adi = df_filtered[[
    "facility", "city", "state", "5_digit_zip_code",
    "adi_nat_2015", "adi_state_2015",
    "adi_nat_2020", "adi_state_2020"
]].drop_duplicates()

# Define a function to update the facility node in Neo4j
def update_facility_adi(tx, row):
    query = """
    MATCH (f:Facility)
    WHERE toLower(f.name) = toLower($facility)
      AND toLower(f.city) = toLower($city)
      AND f.state = $state
      AND f.zip_code STARTS WITH $zip
    SET f.adi_nat_2015 = $adi_nat_2015,
        f.adi_state_2015 = $adi_state_2015,
        f.adi_nat_2020 = $adi_nat_2020,
        f.adi_state_2020 = $adi_state_2020;
    """
    tx.run(query,
           facility=row["facility"],
           city=row["city"],
           state=row["state"],
           zip=row["5_digit_zip_code"],
           adi_nat_2015=row["adi_nat_2015"],
           adi_state_2015=row["adi_state_2015"],
           adi_nat_2020=row["adi_nat_2020"],
           adi_state_2020=row["adi_state_2020"])

# Push the updates to Neo4j
with driver.session() as session:
    print(" Updating Facility nodes with ADI scores...")
    for _, row in tqdm(facility_adi.iterrows(), total=facility_adi.shape[0]):
        session.execute_write(update_facility_adi, row)

print(" ADI scores successfully updated for Facility nodes in Neo4j!")


In [ ]:
# MATCH (f:Facility)
# WHERE toString(f.adi_nat_2015) = "NaN"
#    OR toString(f.adi_state_2015) = "NaN"
#    OR toString(f.adi_nat_2020) = "NaN"
#    OR toString(f.adi_state_2020) = "NaN"
# SET f.adi_nat_2015 = CASE WHEN toString(f.adi_nat_2015) = "NaN" THEN NULL ELSE f.adi_nat_2015 END,
#     f.adi_state_2015 = CASE WHEN toString(f.adi_state_2015) = "NaN" THEN NULL ELSE f.adi_state_2015 END,
#     f.adi_nat_2020 = CASE WHEN toString(f.adi_nat_2020) = "NaN" THEN NULL ELSE f.adi_nat_2020 END,
#     f.adi_state_2020 = CASE WHEN toString(f.adi_state_2020) = "NaN" THEN NULL ELSE f.adi_state_2020 END;
